# Linear Regression

## Table of contents
* [Appendix](#Appendix)


<a id='Top'></a>
One of the most widely used models for regression is known as linear regression. This **asserts
that the response is a linear function of the inputs**. This can be written as follows:
![](img/f1.png)
where $\mathbf{w}^{T}{\mathbf{x}}$ represents the inner or **scalar product** between the input vector $\mathsf{x}$ and the model’s weight vector $\mathsf{w}$ , and $\epsilon$ is the **residual error** between our linear predictions and the true response.
We often assume that $\epsilon$ has a Gaussian or normal distribution. We denote this by $\epsilon\sim\mathcal{N}(\mu,\sigma^2)$ where $\mu$ is the mean and $\sigma^2$ is the variance. When we plot this distribution, we get the well-known bell curve (this is Gaussian pdf with mean 0 and variance 1):
![A Gaussian pdf with mean 0 and variance 1](img/f2.png)
To make the connection between linear regression and Gaussians more explicit, we can rewrite the model in the following form:
![](img/f3.png)

## Algorithm

In [1]:
# NumPy
import numpy as np

# Plotting
import time
from matplotlib import pyplot as plt
from IPython import display
%matplotlib inline

from abc import ABCMeta, abstractmethod

# For testing
from sklearn.datasets.samples_generator import make_blobs
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Scikit-learn Linear Regression for comparing results
import sklearn.linear_model

Let's implements the simplest version of Linear Regfression algorithm, based on ordinary least squares method 

In [2]:
class LinearRegression():
        @abstractmethod
        def __init__(self):
            self.X = None
            self.init_X = None
            self.y = None
            self.coef_ = None
            self.intercept_ = None
            pass
        
        @abstractmethod
        def fit(self, X_train, y_train):
            self.init_X = np.array(X_train)
            self.X = np.array(X_train)
            self.y = np.array(y_train)
            # 1. Initialize weights and add w0 (intercept) to weights:
            self.coef_ = np.random.randn(len(self.X[0]) + 1)
            # 2. Add constant x0=1 to all rows in X:
            self.X = np.insert(self.X, 0, [1], axis=1)
            # One step:
            self.coef_ = np.dot(np.linalg.pinv(self.X), self.y)
            self.intercept_ = self.coef_[0]
        
        @abstractmethod
        def predict(self, usr_X):
            # 0. Do we have matrix X as an input, or just single vector x?
            if np.array(usr_X).ndim == 1: # vector
                self.X = np.array([usr_X[:]])
            else:                         # matrix
                self.X = np.array(usr_X[:])
            # 1. Add constant x0=1 to all rows in X:
            self.X = np.insert(self.X, 0, [1], axis=1)
            # 1. Predict (return dot product):
            return np.dot(self.X, self.coef_)

        @abstractmethod
        def score(self, X_test, y_test, metric='rmse'):
            # 1. Get a prediction:
            y_pred = self.predict(X_test)
            # 2. Calculate score:
            if metric == 'rmse':
                """RMSE"""
                return np.sqrt((np.sum(np.square(y_pred - y_test))) / (len(y_test)))
            elif metric == 'r2':
                "R2"
                y = np.array(y_test)
                return 1 - np.square(np.linalg.norm(y - y_pred)) / np.square(np.linalg.norm(y - y.mean()))
            else:
                return None

In [3]:
# Creating random data 
X, y = make_blobs(n_samples=20, centers=2, n_features=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3)

# --------------------- Testing this class ------------------------ #
print("This Linear regression:")
lr1 = LinearRegression()
lr1.fit(X_train, y_train)
pred1 = lr1.predict(X_test)
print("\tCoefficients (intercept is the first)", lr1.coef_)
print("\tMy RMSE:", lr1.score(X_test, y_test, 'rmse'), " --- Real RMSE:", np.sqrt(mean_squared_error(y_test, pred1)))
print("\tMy R2:", lr1.score(X, y, 'r2'), " --- Real R2:", r2_score(y_test, pred1))

# --------- Comparing with scikit-learn Linear Regression ---------- #
print("-"*40,"\n\nScikit-learn Linear regression:")
lr2 = sklearn.linear_model.LinearRegression()
lr2.fit(X_train, y_train)
pred2 = lr2.predict(X_test)
print("\tIntercept:", lr2.intercept_, "Coefficients:", lr2.coef_)
print("\tSK-learn RMSE:", np.sqrt(mean_squared_error(y_test, pred2)))
print("\tSK-learn R2:", r2_score(y_test, pred2))

This Linear regression:
	Coefficients (intercept is the first) [ 0.30504788 -0.12155051 -0.10398831 -0.02560036]
	My RMSE: 0.123848708789  --- Real RMSE: 0.123848708789
	My R2: 0.962634113168  --- Real R2: 0.889562780786
---------------------------------------- 

Scikit-learn Linear regression:
	Intercept: 0.305047880538 Coefficients: [-0.12155051 -0.10398831 -0.02560036]
	SK-learn RMSE: 0.123848708789
	SK-learn R2: 0.889562780786


## Statistical information

Let us add functionality for viewing statistical info.

In [4]:
class LinearRegression(LinearRegression):
    
        @abstractmethod
        def show_stats(self):
            print('='*50, '\nShowing statistical info for linear regression\n')
            y_pred = self.predict(self.init_X)
            # Residuals sum (should be zero if model contains intercept term):
            RS = sum([y-y_hat for y, y_hat in zip(self.y, y_pred)])
            print("Residuals sum:", RS)
            # Residual sum of squares
            RSS = sum([np.square(y - y_hat) for y, y_hat in zip(self.y, y_pred)])
            print("Residual sum of squares:", RSS)
            # RMSE score:
            print("RMSE score:", np.sqrt(RSS / len(self.y)))
            # Residual Standard Error:
            RSE = np.sqrt(RSS/(len(self.y) - 2))
            print("RSE:", RSE)
            

In [5]:
lr1 = LinearRegression()
lr1.fit(X_train, y_train)
lr1.show_stats()
lr1.coef_

Showing statistical info for linear regression

Residuals sum: 5.55111512313e-15
Residual sum of squares: 0.0947984181462
RMSE score: 0.0822880038758
RSE: 0.0888812026181


array([ 0.30504788, -0.12155051, -0.10398831, -0.02560036])

In [22]:
X = [[0],[2],[2],[5],[5],[9],[9],[9],[9],[10]]
y = [-2,0,2,1,3,1,0,0,1,-1]

"""
X = [3.87, 3.61, 4.33, 3.43, 3.81, 3.83, 3.46, 3.76, 3.50, 3.58, 4.19, 3.78, 3.71, 3.73, 3.78]
y = [4.87, 3.93, 6.46, 3.33, 4.38, 4.70, 3.50, 4.50, 3.58, 3.64, 5.90, 4.43, 4.38, 4.42, 4.25]
"""


'\nX = [3.87, 3.61, 4.33, 3.43, 3.81, 3.83, 3.46, 3.76, 3.50, 3.58, 4.19, 3.78, 3.71, 3.73, 3.78]\ny = [4.87, 3.93, 6.46, 3.33, 4.38, 4.70, 3.50, 4.50, 3.58, 3.64, 5.90, 4.43, 4.38, 4.42, 4.25]\n'

In [24]:
lr1 = LinearRegression()
lr1.fit(X, y)
lr1.coef_

array([ 0.40163934,  0.01639344])

In [16]:
import statsmodels.api as sm
ols = sm.OLS(y, X)
r = ols.fit()
print(r.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.101
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.013
Date:                Sat, 16 Dec 2017   Prob (F-statistic):              0.340
Time:                        07:08:35   Log-Likelihood:                -17.366
No. Observations:                  10   AIC:                             36.73
Df Residuals:                       9   BIC:                             37.03
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0664      0.066      1.006      0.3

/home/konstantin/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1334: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  "anyway, n=%i" % int(n))


<a id='Appendix'></a>
## Appendix
[↑ To the top](#Top)<br><br>
Code for a short simple Linear Regression class:

In [222]:
import numpy as np


class LinearRegression:
    def __init__(self):
        self.X = None
        self.y = None
        self.coef_ = None
        self.intercept_ = None
        pass

    def fit(self, X_train, y_train):
        self.X = np.array(X_train)
        self.y = np.array(y_train)
        # 1. Initialize weights and add w0 (intercept) to weights:
        self.coef_ = np.random.randn(len(self.X[0]) + 1)
        # 2. Add constant x0=1 to all rows in X:
        self.X = np.insert(self.X, 0, [1], axis=1)
        # One step:
        self.coef_ = np.dot(np.linalg.pinv(self.X), self.y)
        self.intercept_ = self.coef_[0]

    def predict(self, usr_X):
        # 0. Do we have matrix X as an input, or just single vector x?
        if np.array(usr_X).ndim == 1:  # vector
            self.X = np.array([usr_X[:]])
        else:  # matrix
            self.X = np.array(usr_X[:])
        # 1. Add constant x0=1 to all rows in X:
        self.X = np.insert(self.X, 0, [1], axis=1)
        # 1. Predict (return dot product):
        return np.dot(self.X, self.coef_)

    def score(self, X_test, y_test, metric='rmse'):
        # 1. Get a prediction:
        y_pred = self.predict(X_test)
        # 2. Calculate score:
        if metric == 'rmse':
            """RMSE"""
            return np.sqrt((np.sum(np.square(y_pred - y_test))) / (len(y_test)))
        elif metric == 'r2':
            "R2"
            y = np.array(y_test)
            return 1 - np.square(np.linalg.norm(y - y_pred)) / np.square(np.linalg.norm(y - y.mean()))
        else:
            return None

For testing: